In [1]:
# current epoch: 20 current mean dice: 0.2194 best mean dice: 0.6352 at epoch 14 for DiceLoss
# current epoch: 20 current mean dice: 0.7359 best mean dice: 0.7712 at epoch 10 for FocalLoss
# current epoch: 20 current mean dice: 0.5530 best mean dice: 0.8096 at epoch 16 for GeneralizedDiceLoss

In [2]:
#!pip install 'monai[all]'

In [3]:
#Thank god-> Starting from Python 3.7, dictionaries maintain the insertion order.

import importlib
import subprocess

def check_and_install_packages(package_list):
    missing_packages = []

    for package in package_list:
        try:
            __import__(package)
        except ImportError:
            missing_packages.append(package)

    if missing_packages:
        print("Installing missing packages:")
        for package in missing_packages:
            subprocess.run(["pip", "install", package])
            print(f"Installed {package}")
    else:
        print("All required packages are already installed.")

# List of packages to check and install if necessary
required_packages = [
    "numpy",
    "matplotlib",
    "pandas",
    #"cv2",
    "imageio",
    "PIL",
    "torch",
    "monai",
    "sys"
]

check_and_install_packages(required_packages)

In [4]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
import os
import cv2 as cv
import imageio
from PIL import Image
import imageio.v3 as iio

In [7]:
#updated_raw_dict, new_analyzed_dict
import PIL

In [8]:
def convert_to_numpy(data):
    if isinstance(data, np.ndarray):
        return data.astype(np.float64)
    elif isinstance(data, PIL.Image.Image):
        return np.array(data).astype(np.float64)
    elif isinstance(data, torch.Tensor):
        return data.numpy().astype(np.float64)
    # Add more conversion cases if needed
    else:
        raise ValueError("Unsupported data format")

def check_and_convert_format(updated_raw_dict, new_analyzed_dict):
    converted_raw_dict = {key: convert_to_numpy(value) for key, value in updated_raw_dict.items()}
    converted_analyzed_dict = {key: convert_to_numpy(value) for key, value in new_analyzed_dict.items()}

    return converted_raw_dict, converted_analyzed_dict


In [9]:
import pickle



In [10]:
import torch
torch.cuda.empty_cache()
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'MAX_SPLIT_SIZE_MB=256'  # Adjust the size as needed

In [11]:
#weight_values = [1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 20.0]  # Example weight values to experiment with

In [12]:
#!pip install tensorboardX
import tensorboardX

In [13]:
import random
# Set a random seed for Python's built-in random module
random_seed = 42
random.seed(random_seed)

# Set a random seed for NumPy
np.random.seed(random_seed)

# Set a random seed for PyTorch on both CPU and CUDA (if available)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [14]:
#!pip install tensorboard

In [15]:
import logging
import os
import sys
import tempfile
from glob import glob

In [16]:
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter
import torch.nn.utils as nn_utils
import torch.autograd as autograd
from torch.optim.lr_scheduler import StepLR

In [17]:
import psutil

In [18]:
import monai

In [19]:
from monai.data import create_test_image_2d, list_data_collate, decollate_batch, DataLoader

In [20]:
from monai.inferers import sliding_window_inference


In [21]:
from monai.metrics import DiceMetric

In [22]:
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    ToTensor,
    AsDiscrete
)
#   DivisiblePadd deleted

In [23]:
from monai.transforms import Transform
#from monai.visualize import plot_2d_or_3d_image

In [24]:
from monai.networks.layers import Norm
from monai.transforms import Resized

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [26]:
#!pip install scikit-learn

In [27]:
from sklearn.model_selection import train_test_split
import random


In [28]:
from PIL import Image

In [29]:


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_files, segmentation_files, transform=None):
        self.input_files = input_files
        self.segmentation_files = segmentation_files
        self.transform = transform

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        input_path = self.input_files[idx]
        segmentation_path = self.segmentation_files[idx]

        # Load input image (TIFF)
        input_image = Image.open(input_path)
        input_image = np.array(input_image, dtype=np.float32)  # Convert to NumPy array

        # Load segmentation image (TIFF)
        segmentation_image = Image.open(segmentation_path)
        segmentation_image = np.array(segmentation_image, dtype=np.float32)  # Convert to NumPy array

        # Initialize variables
        input_data = input_image
        segmentation_data = segmentation_image

        if self.transform is not None:
            # Apply the specified transforms to input_data and segmentation_data
            transformed_data = self.transform({"img": input_data, "seg": segmentation_data})
            input_data, segmentation_data = transformed_data["img"], transformed_data["seg"]

        return {"img": input_data, "seg": segmentation_data}






max_grad_norm = 1.0
# Define a custom transform to threshold the segmentation tensor
class ThresholdSegmentation(Transform):
    def __init__(self, keys, threshold=0):
        super().__init__()
        self.keys = keys
        self.threshold = threshold

    def __call__(self, data):
        for key in self.keys:
            if key == "seg":
                data[key] = (data[key] > self.threshold).float()
        return data

import matplotlib.pyplot as plt
from monai.transforms import ToNumpy




def main():
    log_dir = "D:/Aytekin/CovidMasterThesis/runs/resize_512_generalized_dice_loss_16_256_lr1e2"
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)


    #print(len(updated_raw_dict))
    #print(len(new_analyzed_dict))
    # Check and convert data format only once
    #converted_raw_dict, converted_analyzed_dict = check_and_convert_format(updated_raw_dict, new_analyzed_dict)

    # Remove red for now!!!!
    #converted_raw_dict = {k: v for k, v in converted_raw_dict.items() if 'red' not in k}

    # Remove elements where the key contains "red" for converted_analyzed_dict
    #converted_analyzed_dict = {k: v for k, v in converted_analyzed_dict.items() if 'red' not in k}


    # Check if the dictionaries contain the same length or not, then create train val test:
    #if len(converted_raw_dict) != len(converted_analyzed_dict):
    #    raise ValueError("The lengths of converted_raw_dict and converted_analyzed_dict do not match.")

    #num_images = len(converted_raw_dict)
    #print("Length of converted_raw_dict to be used: {}".format(num_images))
    # Calculate the number of images for training, validation, and test, e.g., using an 80-10-10 split

    #
    #converted_raw_dict.keys()
    
    
    

        # define transforms for image and segmentation
    train_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),

            #DivisiblePadd(keys=["img", "seg"],k=16),

        ]
    )
    val_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),
            #DivisiblePadd(keys=["img", "seg"],k=16),


        ]
    )
    
    
    
    



    # Directory paths for input and segmentation images
    input_image_dir = r"E:\Aytekin\Data\Input"
    segmentation_dir = r"E:\Aytekin\Data\Segmentation"
    test_input_dir = r"E:\Aytekin\Data\Test\Input"
    ignore_dir = r"E:\Aytekin\Data\UniqueIntensityLessThan750"  # New directory to ignore

    # List of strings to exclude from image names
    exclude_strings = ["12_0500", "29_0520", "01_0520", "08_0420", "29_0820", "29_1120", "29_1420", "29_1720", "29_2020", "29_2320"]

    # List all .tif image files in the input and segmentation directories
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]

    # Get the list of files in the "Test Input" folder
    test_input_files = [os.path.splitext(os.path.basename(filename))[0] for filename in os.listdir(test_input_dir) if filename.endswith(".tif")]

    # Exclude images with the same names as those in the "Test Input" folder
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]

    # Extract file names without extensions
    input_image_names = [os.path.splitext(os.path.basename(filename))[0] for filename in input_image_files]
    segmentation_names = [os.path.splitext(os.path.basename(filename))[0] for filename in segmentation_files]

    # Find the common file names between input and segmentation
    common_names = set(input_image_names) & set(segmentation_names)

    # List all .tif image files in the directories again (overwrite previous lists)
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif")]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif")]

    # Exclude images from UniqueIntensityLessThan750 directory
    ignore_files = [os.path.join(ignore_dir, filename) for filename in os.listdir(ignore_dir) if filename.endswith(".tif")]
    ignore_file_names = [os.path.splitext(os.path.basename(filename))[0] for filename in ignore_files]

    # Filter input_image_files and segmentation_files based on common names and ignoring specific names
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] in common_names and os.path.splitext(os.path.basename(filename))[0] not in ignore_file_names]



    # Split the data into training and validation sets
    input_image_train, input_image_val, segmentation_train, segmentation_val = train_test_split(
        input_image_files, segmentation_files, test_size=0.2, random_state=42)
    
    # Print the total number of images used in training and validation
    print(f"Total number of images in training set: {len(input_image_train)}")
    print(f"Total number of images in validation set: {len(input_image_val)}")


    # Create instances of the custom dataset for training and validation
    train_dataset = CustomDataset(input_image_train, segmentation_train, transform=train_transforms)
    val_dataset = CustomDataset(input_image_val, segmentation_val, transform=val_transforms)

    # Define data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=0, collate_fn=list_data_collate, pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=1, num_workers=0, collate_fn=list_data_collate)








    # define dataset, data loader
    #check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #check_loader = DataLoader(check_ds, batch_size=2, num_workers=0, collate_fn=list_data_collate)
    #check_data = monai.utils.misc.first(check_loader)
    #print(check_data["img"].shape, check_data["seg"].shape)
    #print("------\n")
    #print(torch.unique(check_data["seg"]))
    # create a training data loader
    #train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #train_loader = DataLoader(
    #    train_ds,
    #    batch_size=2,
    #    shuffle=False,
    #    num_workers=0,
    #    collate_fn=list_data_collate,
    #    pin_memory=torch.cuda.is_available(),
    #)
    # create a validation data loader
    #val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    #val_loader = DataLoader(val_ds, batch_size=1, num_workers=0, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
    post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    # create UNet, DiceLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    ######## Different Models Can be Tested ##############
    model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=3,
        norm = Norm.BATCH
    ).to(device)
    
    
    ######## Different Models Can be Tested ##############
    #model = monai.networks.nets.UNet(
    #    spatial_dims=2,
    #    in_channels=1,
    #    out_channels=1,
    #    channels=(64, 128, 256, 512, 1024, 2048),
    #    strides=(2, 2, 2, 2, 2),
    #    num_res_units=4,
    #    norm=None  # Remove normalization layers
    #).to(device)
    
    loss_function = monai.losses.GeneralizedDiceLoss(include_background=True, sigmoid=True) #, weight = [1/0.9892240500840984, 1/0.01077594991590156]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2 ) #1e-3
    #optimizer = Novograd(model.parameters(), lr=1e-2, clip=1.0)
    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter(log_dir=log_dir)


    autograd.set_detect_anomaly(True)
    scaler = GradScaler()

    # Define the step size and gamma (decay factor)
    step_size = 1800  # Adjust this according to your training schedule
    gamma = 0.5     # Adjust this according to your preference

    # Create a learning rate scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # Create a DiceMetric instance for training
    train_dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

    for epoch in range(20):
        print("-" * 20)
        print(f"epoch {epoch + 1}/{20}")
        model.train()
        epoch_loss = 0
        step = 0
        # Clear the DiceMetric for the new epoch
        train_dice_metric.reset()
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

            # Calculate the Dice score during training
            post_outputs = post_trans(outputs)
            train_dice_metric(y_pred=post_outputs, y=labels)

            scaler.scale(loss).backward()
            #scaler.unscale_(optimizer)  # Unscales the gradients of optimizer's assigned params in-place
            ############## Gradient Clipping Part ###################
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Add this line to clip gradients
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            epoch_len = len(train_dataset) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)




        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        # Calculate the average Dice score for the epoch
        train_dice_score = train_dice_metric.aggregate().item()
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}, train_dice: {train_dice_score:.4f}")
        writer.add_scalar("train_dice", train_dice_score, epoch + 1)


        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        scheduler.step()
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                    roi_size = (96, 96)
                    sw_batch_size = 4
                    val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                    val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)

                    for i in range(len(val_images)):
                        input_image = val_images[i:i + 1]  # Get the i-th image
                        ground_truth = val_labels[i:i + 1]  # Get the i-th ground truth
                        prediction = val_outputs[i:i + 1]  # Get the i-th prediction

                        # Perform intensity scaling if needed
                        #a_min = 0.0
                        #a_max = 255.0
                        #b_min = 0.0
                        #b_max = 1.0

                        if isinstance(prediction, list):
                            prediction = prediction[0]
                        
                        
                        #input_image_np = input_image.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #ground_truth_np = ground_truth.detach().cpu().numpy()  # Convert tensor to NumPy array
                        #prediction_np = prediction.detach().cpu().numpy()  # Convert tensor to NumPy array
                        
                        #input_image_np = (input_image_np - input_image_np.min()) * (255.0 / (input_image_np.max() - input_image_np.min()))
                        #ground_truth_np = (ground_truth_np - ground_truth_np.min()) * (255.0 / (ground_truth_np.max() - ground_truth_np.min()))
                        #prediction_np = (prediction_np - prediction_np.min()) * (255.0 / (prediction_np.max() - prediction_np.min()))
                        
                        
                        writer.add_image('Validation/Input', input_image, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/GroundTruth', ground_truth, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/Prediction', prediction, global_step=epoch,dataformats='CHW')


                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "512green_alldata_generalized_diceloss_epoch20_16_256_lr1e2.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_dice", metric, epoch + 1)
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label




    autograd.set_detect_anomaly(False)
    writer.close()






if __name__ == "__main__":
    main()

MONAI version: 1.2.0
Numpy version: 1.26.0
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.22.0
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details ab

252/820, train_loss: 0.1995
253/820, train_loss: 0.1831
254/820, train_loss: 0.2150
255/820, train_loss: 0.2091
256/820, train_loss: 0.2265
257/820, train_loss: 0.1838
258/820, train_loss: 0.1852
259/820, train_loss: 0.2152
260/820, train_loss: 0.2005
261/820, train_loss: 0.1988
262/820, train_loss: 0.2035
263/820, train_loss: 0.1653
264/820, train_loss: 0.1596
265/820, train_loss: 0.1914
266/820, train_loss: 0.1938
267/820, train_loss: 0.1936
268/820, train_loss: 0.1786
269/820, train_loss: 0.2115
270/820, train_loss: 0.1688
271/820, train_loss: 0.1735
272/820, train_loss: 0.1861
273/820, train_loss: 0.1862
274/820, train_loss: 0.1686
275/820, train_loss: 0.2017
276/820, train_loss: 0.1788
277/820, train_loss: 0.2578
278/820, train_loss: 0.2248
279/820, train_loss: 0.1508
280/820, train_loss: 0.1664
281/820, train_loss: 0.2023
282/820, train_loss: 0.2177
283/820, train_loss: 0.2256
284/820, train_loss: 0.1584
285/820, train_loss: 0.1865
286/820, train_loss: 0.2025
287/820, train_loss:

545/820, train_loss: 0.1589
546/820, train_loss: 0.1642
547/820, train_loss: 0.1501
548/820, train_loss: 0.1733
549/820, train_loss: 0.1491
550/820, train_loss: 0.1810
551/820, train_loss: 0.1589
552/820, train_loss: 0.1309
553/820, train_loss: 0.1188
554/820, train_loss: 0.1412
555/820, train_loss: 0.1522
556/820, train_loss: 0.1434
557/820, train_loss: 0.1832
558/820, train_loss: 0.1330
559/820, train_loss: 0.1177
560/820, train_loss: 0.1373
561/820, train_loss: 0.1472
562/820, train_loss: 0.1407
563/820, train_loss: 0.1438
564/820, train_loss: 0.1339
565/820, train_loss: 0.1665
566/820, train_loss: 0.1773
567/820, train_loss: 0.1561
568/820, train_loss: 0.1302
569/820, train_loss: 0.1526
570/820, train_loss: 0.1466
571/820, train_loss: 0.1484
572/820, train_loss: 0.1387
573/820, train_loss: 0.1381
574/820, train_loss: 0.1180
575/820, train_loss: 0.1420
576/820, train_loss: 0.1600
577/820, train_loss: 0.1472
578/820, train_loss: 0.1485
579/820, train_loss: 0.1425
580/820, train_loss:

14/820, train_loss: 0.1316
15/820, train_loss: 0.1182
16/820, train_loss: 0.1169
17/820, train_loss: 0.1488
18/820, train_loss: 0.1442
19/820, train_loss: 0.1543
20/820, train_loss: 0.1579
21/820, train_loss: 0.1299
22/820, train_loss: 0.1369
23/820, train_loss: 0.1350
24/820, train_loss: 0.1478
25/820, train_loss: 0.1740
26/820, train_loss: 0.1117
27/820, train_loss: 0.1492
28/820, train_loss: 0.1658
29/820, train_loss: 0.1314
30/820, train_loss: 0.1153
31/820, train_loss: 0.1161
32/820, train_loss: 0.1545
33/820, train_loss: 0.1464
34/820, train_loss: 0.1385
35/820, train_loss: 0.1509
36/820, train_loss: 0.1492
37/820, train_loss: 0.1474
38/820, train_loss: 0.1378
39/820, train_loss: 0.1423
40/820, train_loss: 0.1197
41/820, train_loss: 0.1582
42/820, train_loss: 0.1550
43/820, train_loss: 0.1431
44/820, train_loss: 0.1397
45/820, train_loss: 0.1463
46/820, train_loss: 0.1565
47/820, train_loss: 0.1243
48/820, train_loss: 0.1312
49/820, train_loss: 0.1322
50/820, train_loss: 0.1375
5

310/820, train_loss: 0.1554
311/820, train_loss: 0.1633
312/820, train_loss: 0.1429
313/820, train_loss: 0.1482
314/820, train_loss: 0.1295
315/820, train_loss: 0.1500
316/820, train_loss: 0.1324
317/820, train_loss: 0.1431
318/820, train_loss: 0.1322
319/820, train_loss: 0.1149
320/820, train_loss: 0.1490
321/820, train_loss: 0.1412
322/820, train_loss: 0.1267
323/820, train_loss: 0.1439
324/820, train_loss: 0.1556
325/820, train_loss: 0.1320
326/820, train_loss: 0.1449
327/820, train_loss: 0.1264
328/820, train_loss: 0.1191
329/820, train_loss: 0.1361
330/820, train_loss: 0.1270
331/820, train_loss: 0.1205
332/820, train_loss: 0.1335
333/820, train_loss: 0.1551
334/820, train_loss: 0.1473
335/820, train_loss: 0.1491
336/820, train_loss: 0.1323
337/820, train_loss: 0.1486
338/820, train_loss: 0.1700
339/820, train_loss: 0.1657
340/820, train_loss: 0.1336
341/820, train_loss: 0.1233
342/820, train_loss: 0.1296
343/820, train_loss: 0.1402
344/820, train_loss: 0.1456
345/820, train_loss:

...

514/820, train_loss: 0.1109
515/820, train_loss: 0.1478
516/820, train_loss: 0.1240
517/820, train_loss: 0.1219
518/820, train_loss: 0.1261
519/820, train_loss: 0.1197
520/820, train_loss: 0.1163
521/820, train_loss: 0.0981
522/820, train_loss: 0.1097
523/820, train_loss: 0.1146
524/820, train_loss: 0.1139
525/820, train_loss: 0.0891
526/820, train_loss: 0.1274
527/820, train_loss: 0.1263
528/820, train_loss: 0.1461
529/820, train_loss: 0.1145
530/820, train_loss: 0.1117
531/820, train_loss: 0.1086
532/820, train_loss: 0.1213
533/820, train_loss: 0.1132
534/820, train_loss: 0.1095
535/820, train_loss: 0.1243
536/820, train_loss: 0.1181
537/820, train_loss: 0.0954
538/820, train_loss: 0.1151
539/820, train_loss: 0.1222
540/820, train_loss: 0.1415
541/820, train_loss: 0.1153
542/820, train_loss: 0.1154
543/820, train_loss: 0.1121
544/820, train_loss: 0.1248
545/820, train_loss: 0.1070
546/820, train_loss: 0.1218
547/820, train_loss: 0.1236
548/820, train_loss: 0.0968
549/820, train_loss:

807/820, train_loss: 0.1101
808/820, train_loss: 0.1084
809/820, train_loss: 0.1130
810/820, train_loss: 0.1047
811/820, train_loss: 0.1233
812/820, train_loss: 0.1397
813/820, train_loss: 0.1273
814/820, train_loss: 0.1023
815/820, train_loss: 0.0927
816/820, train_loss: 0.1152
817/820, train_loss: 0.1245
818/820, train_loss: 0.1259
819/820, train_loss: 0.1101
820/820, train_loss: 0.1078
821/820, train_loss: 0.1484
epoch 16 average loss: 0.1159, train_dice: 0.8827
epoch 16 average loss: 0.1159
current epoch: 16 current mean dice: 0.8425 best mean dice: 0.8696 at epoch 8
--------------------
epoch 17/20
1/820, train_loss: 0.1369
2/820, train_loss: 0.1157
3/820, train_loss: 0.1212
4/820, train_loss: 0.1163
5/820, train_loss: 0.1156
6/820, train_loss: 0.1148
7/820, train_loss: 0.1101
8/820, train_loss: 0.1082
9/820, train_loss: 0.1160
10/820, train_loss: 0.1427
11/820, train_loss: 0.1314
12/820, train_loss: 0.1334
13/820, train_loss: 0.1210
14/820, train_loss: 0.1204
15/820, train_loss: 

276/820, train_loss: 0.1249
277/820, train_loss: 0.1117
278/820, train_loss: 0.1064
279/820, train_loss: 0.1199
280/820, train_loss: 0.1345
281/820, train_loss: 0.1051
282/820, train_loss: 0.1288
283/820, train_loss: 0.1199
284/820, train_loss: 0.1254
285/820, train_loss: 0.1069
286/820, train_loss: 0.0895
287/820, train_loss: 0.1361
288/820, train_loss: 0.1026
289/820, train_loss: 0.1131
290/820, train_loss: 0.1583
291/820, train_loss: 0.0980
292/820, train_loss: 0.1157
293/820, train_loss: 0.1097
294/820, train_loss: 0.1222
295/820, train_loss: 0.1229
296/820, train_loss: 0.1035
297/820, train_loss: 0.1108
298/820, train_loss: 0.1229
299/820, train_loss: 0.1249
300/820, train_loss: 0.1230
301/820, train_loss: 0.0967
302/820, train_loss: 0.1099
303/820, train_loss: 0.1170
304/820, train_loss: 0.1155
305/820, train_loss: 0.1304
306/820, train_loss: 0.1190
307/820, train_loss: 0.1152
308/820, train_loss: 0.1111
309/820, train_loss: 0.1187
310/820, train_loss: 0.1092
311/820, train_loss:

C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\torch\autograd\__init__.py:200: UserWarning: Error detected in ConvolutionBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start
    self.io_loop.start()
  File "C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\ge59gor\AppData\Local\anaconda3\en

RuntimeError: Function 'ConvolutionBackward0' returned nan values in its 1th output.

In [ ]:
!tensorboard --logdir=D:/Aytekin/CovidMasterThesis/runs/resize_512_generalized_dice_loss_16_256_lr1e2 --port=6032

In [ ]:
print(a)

In [ ]:
# Started at 8:53 17.12.2023

In [ ]:
# import os
#os.system("pkill -f 'tensorboard'")

In [ ]:
#%load_ext tensorboard

In [ ]:
#!pip install --upgrade tensorboard

In [ ]:
# 13 saatte 1 epoch.